# Imports y funciones

In [ ]:
# # 📦 Importar librerías necesarias
import pyspark
import pandas as pd
import numpy as np
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os

In [ ]:
import kagglehub

# Descargar el dataset desde Kaggle Hub
path = kagglehub.dataset_download("davidcariboo/player-scores")

print("Path to dataset files:", path)

# Lista los archivos descargados
for root, _, files in os.walk(path):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
spark = SparkSession.builder.appName('tfm_2025').getOrCreate()

Funciones utiles para la limpieza

In [ ]:
def mostrar_sumario(df):
    """
    Muestra un sumario del dataset.
    Explicacion:
        - Usa la funcion 'describe()' para mostrar las siguientes metricas:
            - El conteo de filas por columna.
            - La media de los valores de las columnas numericas (muestra NULL si no es numerica).
            - La desviacion  estandar de los valores de las columnas numericas (muestra NULL si no es numerica).
            - El valor minimo de cada columna.
            - El valor maximo de cada columna.
        - Devuelve el DataFrame con las metricas.
    Args:
        df: DataFrame a analizar

    Returns:
        Un DataFrame con las métricas del dataset.
    """
    return df.summary()

In [ ]:
def contar_nulos_por_columna(df):
    """
    Calcula el numero de valores nulos en cada columna del DataFrame.
    Explicacion:
        - Itera sobre cada columna del DataFrame utilizando 'df.columns'.
        - Usa la funcion 'expr()' que utiliza una expresion SQL para contar los valores nulos en cada columna.
        - Asigna '1' si el valor es nulo y '0' si no es nulo.
        - Renombra la columna con '.alias(c)' para mantener el nombre original.
        - Selecciona y aplica la transformacion en todas las columnas dinamicamente con 'df.select([...])'.
        - Devuelve el DataFrame con los conteos de nulos.
    Args:
        df: DataFrame a analizar

    Returns:
        Un DataFrame con las mismas columnas del DataFrame original
        y una fila que indica el numero de valores nulos por columna.
    """
    # df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]).show()
    return df.select([expr(f"sum(case when {c} is null then 1 else 0 end)").alias(c) for c in df.columns])

In [ ]:
def duplicados_por_columna(df, columna):
    """
    Muestra los valores duplicados y su conteo para una columna especifica.
    Explicacion:
        - Agrupa el DataFrame por la columna especificada.
        - Cuenta el numero de ocurrencias de cada valor en la columna.
        - Filtra para ver los valores que aparecen mas de una vez.
    Args:
        df: DataFrame a analizar.
        columna: Nombre de la columna a verificar.
    """
    df.groupBy(columna).count().filter(col("count") > 1).show(truncate=False)

# Archivo "clubs.csv"

In [ ]:
# Cargar el csv
df_clubs = spark.read.csv("/root/.cache/kagglehub/datasets/davidcariboo/player-scores/versions/602/clubs.csv", header=True, inferSchema=True)

# Ver las primeras filas
df_clubs.show(10)
df_clubs.printSchema()
df_clubs.columns

In [ ]:
mostrar_sumario(df_clubs).show()

In [ ]:
contar_nulos_por_columna(df_clubs).show()

In [ ]:
duplicados_por_columna(df_clubs, "club_id")

In [ ]:
# Comparar numero de filas totales con numero de nulos en 'total_market_value'
print(f"Filas totales: {df_clubs.count()}")
print(f"Nulos en la columna 'total_market_value': {contar_nulos_por_columna(df_clubs).select('total_market_value').first()[0]}")

In [ ]:
# La columna 'total_market_value' esta completamente vacia, podemos eliminarla
df_clubs = df_clubs.drop("total_market_value")
df_clubs.show()

In [ ]:
# Ver nulos de la columna 'average_age'
df_clubs.filter(col('average_age').isNull()).show()

In [ ]:
# Calcular la media de 'average_age' por liga (excluyendo nulos) y redondear a 1 decimal
avg_age_by_league_df = df_clubs.filter(col("average_age").isNotNull()).groupBy("domestic_competition_id").agg(round(avg("average_age"), 1).alias("avg_age_by_league"))

# Unir la nueva columna al dataset
df_clubs = df_clubs.join(avg_age_by_league_df, on="domestic_competition_id", how="left_outer")

# Ver nueva columna
df_clubs.select("domestic_competition_id", "avg_age_by_league").show()

In [ ]:
# Sustituir nulos por 'avg_age_by_league'
df_clubs = df_clubs.withColumn(
    "average_age",
    when(col("average_age").isNull(), col("avg_age_by_league")).otherwise(col("average_age"))
)

# Eliminar columna de media por liga
df_clubs = df_clubs.drop("avg_age_by_league")

# Comprobar de nuevo los nulos despues de la sustitucion
df_clubs.filter(col('average_age').isNull()).show()

In [ ]:
# Ver nulos de la columna 'foreigners_percentage'
df_clubs.filter(col('foreigners_percentage').isNull()).show()

In [ ]:
# Calcular la media de 'foreigners_percentage' por liga (excluyendo nulos) y redondear a 1 decimal
avg_foreigners_percentage_df = df_clubs.filter(col("foreigners_percentage").isNotNull()).groupBy("domestic_competition_id").agg(round(avg("foreigners_percentage"), 1).alias("avg_foreigners_percentage_by_league"))

# Unir la nueva columna al dataset
df_clubs = df_clubs.join(avg_foreigners_percentage_df, on="domestic_competition_id", how="left_outer")

# Ver nueva columna
df_clubs.select("domestic_competition_id", "avg_foreigners_percentage_by_league").show()

In [ ]:
# Sustituir nulos por 'avg_foreigners_percentage_by_league'
df_clubs = df_clubs.withColumn(
    "foreigners_percentage",
    when(col("foreigners_percentage").isNull(), col("avg_foreigners_percentage_by_league")).otherwise(col("foreigners_percentage"))
)

# Eliminar tabla de media por liga
df_clubs = df_clubs.drop("avg_foreigners_percentage_by_league")

# Comprobar de nuevo los nulos despues de la sustitucion
contar_nulos_por_columna(df_clubs).show()

In [ ]:
# Eliminamos las columnas 'filename' y 'url' porque no son relevantes para nuestro analisis
# Tambien eliminamos la columna 'coach_name' porque esa informacion ya esta en otra tabla
df_clubs = df_clubs.drop("filename", "url", "coach_name")
df_clubs.show(10)

# Archivo "competitions.csv"

In [ ]:
# Cargar el csv
df_competitions = spark.read.csv("/root/.cache/kagglehub/datasets/davidcariboo/player-scores/versions/602/competitions.csv", header=True, inferSchema=True)

# Ver las primeras filas
df_competitions.show(10)
df_competitions.printSchema()
df_competitions.columns

In [ ]:
mostrar_sumario(df_competitions).show()

In [ ]:
contar_nulos_por_columna(df_competitions).show()

In [ ]:
duplicados_por_columna(df_competitions, "competition_id")

In [ ]:
# Ver nulos en 'country_name'
df_competitions.filter(col('country_name').isNull()).show()

In [ ]:
# Las filas con 'country_id'= -1 y 'country_name'= null son de competiciones internacionales
# Sustituimos 'country_name' por 'International', 'country_id' por '1' y 'domestic_league_code' por 'INT'
df_competitions = df_competitions.withColumn(
    "country_name",
    when(col("country_id") == -1, "International").otherwise(col("country_name"))
)

df_competitions = df_competitions.withColumn(
    "country_id",
    when(col("country_id") == -1, 1).otherwise(col("country_id"))
)

df_competitions = df_competitions.withColumn(
    "domestic_league_code",
    when(col("country_id") == 1, "INT").otherwise(col("domestic_league_code"))
)

contar_nulos_por_columna(df_competitions).show()

In [ ]:
# Eliminar la columna 'url' porque no es relevante para nuestro analisis
df_competitions = df_competitions.drop("url")
df_competitions.show()

# Archivo "game_events.csv"

In [ ]:
# Cargar el csv
df_game_events = spark.read.csv("/kaggle/input/player-scores/game_events.csv", header=True, inferSchema=True)

# Ver las primeras filas
df_game_events.show(10)
df_game_events.printSchema()
df_game_events.columns

In [ ]:
mostrar_sumario(df_game_events).show()

In [ ]:
contar_nulos_por_columna(df_game_events).show()

In [ ]:
duplicados_por_columna(df_game_events, "game_event_id")

In [ ]:
# Mostrar los diferentes tipos de eventos
df_game_events.select("type").distinct().show()

In [ ]:
# Ver nulos de la columna 'description'
df_game_events.filter(col('description').isNull()).show()

In [ ]:
# Ver valores unicos de 'description'
df_game_events.select("description").distinct().show(truncate=False)

In [ ]:
# Debido a la gran cantidad de valores nulos y formatos confusos,
# consideramos que es mejor eliminar la columna 'description', pues ya tenemos el tipo de jugada en la columna 'type'
df_game_events = df_game_events.drop("description")
df_game_events.show()

Los valores vacíos en las columnas 'player_in_id' y 'player_assist_id' tienen sentido ya que dependen del tipo de evento; si el evento es 'Substitutions', sólo hace falta el 'player_in_id'; si el evento es 'Goals', sólo hace falta el 'player_assist_id'.

# Archivo "game_lineups.csv"

In [ ]:
# Este archivo tiene problemas cargando el esquema, decidimos crear un esquema personalizado para este archivo
custom_schema = StructType([
    StructField("game_lineups_id", StringType(), True),
    StructField("date", DateType(), True),
    StructField("game_id", IntegerType(), True),
    StructField("player_id", IntegerType(), True),
    StructField("club_id", IntegerType(), True),
    StructField("player_name", StringType(), True),
    StructField("type", StringType(), True),
    StructField("position", StringType(), True),
    StructField("number", StringType(), True),
    StructField("team_captain", IntegerType(), True)
])
# Cargar el csv
df_game_lineups = spark.read.csv("/kaggle/input/player-scores/game_lineups.csv", header=True, schema=custom_schema)

# Ver las primeras filas
df_game_lineups.show(10)
df_game_lineups.printSchema()
df_game_lineups.columns

In [ ]:
mostrar_sumario(df_game_lineups).show()

In [ ]:
contar_nulos_por_columna(df_game_lineups).show()

In [ ]:
duplicados_por_columna(df_game_lineups, "game_lineups_id")

In [ ]:
# ver filas con game_lineups_id = 77
df_game_lineups.filter(col('game_lineups_id') == 77).show()

In [ ]:
# Parece que algunas filas se introdujeron incorrectamente y no tienen valores
# Eliminaremos esas filas

duplicated_ids_df = df_game_lineups.groupBy("game_lineups_id").count().filter(col("count") > 1).select("game_lineups_id")

# Convertir el DataFrame de IDs duplicados a una lista de strings
# Esto es necesario para usarlo en una operacion de filtrado
duplicated_ids_list = [row.game_lineups_id for row in duplicated_ids_df.collect()]

# Eliminar las filas con IDs duplicados del DataFrame original
df_game_lineups_cleaned = df_game_lineups.filter(~col("game_lineups_id").isin(duplicated_ids_list))

# Mostrar el conteo de filas antes y despues para verificar la eliminacion
print(f"Filas antes de eliminar duplicados: {df_game_lineups.count()}")
print(f"Filas despues de eliminar duplicados: {df_game_lineups_cleaned.count()}")

In [ ]:
# Ver nulos despues de eliminar duplicados
contar_nulos_por_columna(df_game_lineups_cleaned).show()

In [ ]:
# Ver nulos de la columna 'team_captain'
df_game_lineups_cleaned.filter(col('team_captain').isNull()).show()

In [ ]:
# ver valores vacios de number
df_game_lineups_cleaned.filter(col('number') == '"').show()

In [ ]:
# Algunos valores de 'team_captain' estan nulos y 'number' tiene varios vacios
# Sustituimos los nulos de 'team_captain' y los vacios de 'number' por 0
df_game_lineups_cleaned = df_game_lineups_cleaned.withColumn(
    "team_captain",
    when(col("team_captain").isNull(), 0).otherwise(col("team_captain"))
)
df_game_lineups_cleaned = df_game_lineups_cleaned.withColumn(
    "number",
    when(col("number") == '"', 0).otherwise(col("number"))
)
# Ver nulos de nuevo
contar_nulos_por_columna(df_game_lineups_cleaned).show()

In [ ]:
# Ver nulos en la columna 'position'
df_game_lineups_cleaned.filter(col('position').isNull()).show()

In [ ]:
# Sustituir posiciones nulas por 'Unkown'
df_game_lineups_cleaned = df_game_lineups_cleaned.withColumn(
    "position",
    when(col("position").isNull(), "Unknown").otherwise(col("position"))
)
df_game_lineups_cleaned.show()

# Archivo "appearances.csv"

In [ ]:
# Cargar el csv
df_appearances = spark.read.csv("/kaggle/input/player-scores/appearances.csv", header=True, inferSchema=True)

# Ver las primeras filas
df_appearances.show(10)
df_appearances.printSchema()
df_appearances.columns

In [ ]:
mostrar_sumario(df_appearances).show()

In [ ]:
contar_nulos_por_columna(df_appearances).show()

In [ ]:
duplicados_por_columna(df_appearances, "appearance_id")

In [ ]:
# Ver nulos de la columna 'player_name'
df_appearances.filter(col('player_name').isNull()).show()

In [ ]:
# Ver valores en 'player_current_club_id'= -1
df_appearances.filter(col('player_current_club_id') == -1).show()

In [ ]:
# Existen 6 jugadores sin nombre y con 'player_current_club_id'= -1, son los unicos valores con este id
# Creemos que es un fallo y decidimos eliminarlos
df_appearances = df_appearances.filter(col('player_current_club_id') != -1)
# Ver nulos de nuevo
contar_nulos_por_columna(df_appearances).show()
df_appearances.show()

# Archivo "player_valuations.csv"

In [ ]:
# Cargar el csv
df_player_valuations = spark.read.csv("/kaggle/input/player-scores/player_valuations.csv", header=True, inferSchema=True)

# Ver las primeras filas
df_player_valuations.show(10)
df_player_valuations.printSchema()
df_player_valuations.columns

In [ ]:
mostrar_sumario(df_player_valuations).show()

In [ ]:
contar_nulos_por_columna(df_player_valuations).show()

In [ ]:
duplicados_por_columna(df_player_valuations, "date")

# Archivo "games.csv"

In [ ]:
# Cargar el csv
df_games = spark.read.csv("/kaggle/input/player-scores/games.csv", header=True, inferSchema=True)

# Ver las primeras filas
df_games.show(10)
df_games.printSchema()
df_games.columns

In [ ]:
mostrar_sumario(df_games).show()

In [ ]:
contar_nulos_por_columna(df_games).show()

In [ ]:
duplicados_por_columna(df_games, "game_id")

In [ ]:
# Ver nulos de la tablas 'home_club_id' y 'away_club_id'
df_games.filter(col('home_club_id').isNull()).show()
df_games.filter(col('away_club_id').isNull()).show()

In [ ]:
# Parece que los partidos sin club_id son de fechas futuras, deberian de eliminarse
df_games = df_games.withColumn('date', to_date(col('date'), 'yyyy-MM-dd')).filter(col('date') <= current_date())
# Ver nulos de nuevo
contar_nulos_por_columna(df_games).show()

In [ ]:
# Ver nulos columnas de 'club_position'
df_games.filter(col('home_club_position').isNull()).show()
df_games.filter(col('away_club_position').isNull()).show()

In [ ]:
# Parece que cuando la competicion es algun tipo de copa, los valores de posicion estan nulos porque las copas no usan ese tipo de ranking

# Convertimos las columnas a string y le añadimos un valor 'Sin Ranking' para estos casos
df_games = df_games.withColumn("home_club_position", col("home_club_position").cast(StringType()))
df_games = df_games.withColumn("away_club_position", col("away_club_position").cast(StringType()))

# La logica toma los casos en los que la position sea nulo y la competicion contenga 'cup' sin tener en cuenta mayusculas, o sea de tipo 'other'
df_games = df_games.withColumn(
    "home_club_position",
    when((lower(col("competition_type")).contains("cup") | (col("competition_type") == "other")) & col("home_club_position").isNull(), "Sin Ranking")
    .otherwise(col("home_club_position"))
)
df_games = df_games.withColumn(
    "away_club_position",
    when((lower(col("competition_type")).contains("cup") | (col("competition_type") == "other")) & col("away_club_position").isNull(), "Sin Ranking")
    .otherwise(col("away_club_position"))
)

# Ver nulos despues de la sustitucion
df_games.filter(col('home_club_position').isNull()).show()
df_games.filter(col('away_club_position').isNull()).show()

In [ ]:
# Parece que aun quedan nulos en position que no son de copas
# Sustituimos los valores nulos restantes por 'Unkown'
df_games = df_games.withColumn(
    "home_club_position",
    when(col("home_club_position").isNull(), "Unknown").otherwise(col("home_club_position"))
)
df_games = df_games.withColumn(
    "away_club_position",
    when(col("away_club_position").isNull(), "Unknown").otherwise(col("away_club_position"))
)
# Ver todos los nulos
contar_nulos_por_columna(df_games).show()

In [ ]:
# Sustituir valores nulos en 'home_club_manager_name', 'away_club_manager_name', 'stadium' y 'referee' por 'Unkown'
df_games = df_games.fillna({
    "home_club_manager_name": "Unknown",
    "away_club_manager_name": "Unknown",
    "stadium": "Unknown",
    "referee": "Unknown"
})
# Comprobar de nuevo los nulos despues de la sustitucion
contar_nulos_por_columna(df_games).show()

In [ ]:
# Ver nulos en columnas 'attendance'
df_games.filter(col('attendance').isNull()).show()

In [ ]:
# Sustituir los valores nulos por la capacidad del estadio
# La capacidad del estadio esta en 'df_clubs'
df_clubs_capacity = df_clubs.select("stadium_name", "stadium_seats")

# Unir la nueva columna al dataset
df_games = df_games.join(df_clubs_capacity, df_games.stadium == df_clubs_capacity.stadium_name, "left")

# Mostrar dataframe despues de la union
df_games.show()

In [ ]:
# Rellenar los nulos en 'attendance' usando la capacidad del estadio correspondiente
df_games = df_games.withColumn(
    "attendance",
    when(col("attendance").isNull(), col("stadium_seats")).otherwise(col("attendance"))
)

# Eliminar las columna temporales
df_games = df_games.drop("stadium_name", "stadium_seats")

# Comprobar los nulos en 'attendance' despues de la sustitucion
df_games.filter(col('attendance').isNull()).show()

In [ ]:
# Aun quedan valores nulos en 'attendance', probablemente sean de estadios que tengan de nombre 'Unkown'
# Sustituir esos valores de 'attendance' por 0
df_games = df_games.withColumn(
    "attendance",
    when(col("attendance").isNull(), 0).otherwise(col("attendance"))
)
# Ver nulos en todas las columnas
contar_nulos_por_columna(df_games).show()

In [ ]:
# Ver valores nulos en las columnas de formaciones
df_games.filter(col('home_club_formation').isNull()).show()
df_games.filter(col('away_club_formation').isNull()).show()

In [ ]:
# Ver valores unicos de las columnas de formaciones
df_games.select("home_club_formation").distinct().show(truncate=False)
df_games.select("away_club_formation").distinct().show(truncate=False)

Las columnas de 'home_club_formation' y 'home_club_formation' tienen muchos nulos y no tienen todas el formato correcto.

Decidimos definir una funcion para aplicarles a todas un formato uniforme.

In [ ]:
def formatear_formaciones(column_name):
    """
    Construye una expresion Spark SQL para limpiar, validar el formato, y formatear una columna de formacion,
    reemplazando con "Unknown" si no cumple el formato.
    Explicacion:
      - Limpia caracteres no deseados en los valores (solo admite numeros y guiones).
      - Usa regexp_extract() para verificar el formato: de 3 a 4 digitos separados por guiones.
      - Si el patron coincide, usa la cadena limpiada; de lo contrario, usa "Unknown".
    Args:
        column_name (str): Nombre de la columna a formatear.

    Returns:
        Expresion Spark SQL para formatear la columna.
    """

    cleaned_formation = regexp_replace(col(column_name), "[^0-9-]", "")

    pattern = r"^(\d+)-(\d+)-(\d+)$|^(\d+)-(\d+)-(\d+)-(\d+)$"
    matches_pattern = regexp_extract(cleaned_formation, pattern, 0) != ""

    final_formation = when(matches_pattern, cleaned_formation).otherwise("Unknown")

    return final_formation

In [ ]:
# Aplicar la funcion de formateo
df_games = df_games.withColumn("home_club_formation", formatear_formaciones("home_club_formation"))
df_games = df_games.withColumn("away_club_formation", formatear_formaciones("away_club_formation"))

# Verificar nulos de nuevo
contar_nulos_por_columna(df_games).show()

# Ver si el formato se aplico correctamente
df_games.select("home_club_formation").distinct().show(truncate=False)
df_games.select("away_club_formation").distinct().show(truncate=False)

In [ ]:
# Hay valores nulos en 'home_club_name' y 'away_club_name', como ya tenemos el 'club_id' decidimos eliminar los nombres de esta tabla
df_games = df_games.drop("home_club_name", "away_club_name")
contar_nulos_por_columna(df_games).show()

In [ ]:
# Eliminamos tambien las columnas 'url' y 'aggregate' porque no son relevantes para nuestro analisis
df_games = df_games.drop("url", "aggregate")
df_games.show()

# Archivo "players.csv"

In [ ]:
# Cargar el csv
df_players = spark.read.csv("/kaggle/input/player-scores/players.csv", header=True, inferSchema=True)

# Ver las primeras filas
df_players.show(10)
df_players.printSchema()
df_players.columns

In [ ]:
mostrar_sumario(df_players).show()

In [ ]:
contar_nulos_por_columna(df_players).show()

In [ ]:
duplicados_por_columna(df_players, "player_id")

In [ ]:
# Ver nulos de la columna 'contract_expiration_date'
df_players.filter(col('contract_expiration_date').isNull()).show()

In [ ]:
# Ver valores unicos de la columna 'last_season'
df_players.select('last_season').distinct().show()

In [ ]:
# A primera vista, parece que los valores nulos de la columna 'contract_expiration_date' representan contratos que siguen en vigor
# Pero tras corroborar con Internet, vimos que tambien hay nulos contratos que ya finalizaron
# Decidimos eliminar la columna 'contract_expiration_date' por no tener informacion fiable y tambien la columna 'agent_name' que esta relacionada con ella
df_players = df_players.drop("contract_expiration_date")
df_players = df_players.drop("agent_name")
df_players.show(10)

In [ ]:
# Ver todos los valores de la columna 'foot'
df_players.select('foot').distinct().show()

In [ ]:
# Sustituir valores nulos de 'foot' por 'sin especificar'
df_players = df_players.withColumn(
    "foot",
    when(col("foot").isNull(), "sin especificar").otherwise(col("foot"))
)

In [ ]:
# Ver valores nulos de la columna 'market_value_in_eur'
df_players.filter(col('market_value_in_eur').isNull()).show()

In [ ]:
# Calcular la mediana del valor de mercado (excluyendo nulos y ceros)
mediana_market_value = df_players.filter(
    col("market_value_in_eur").isNotNull() & (col("market_value_in_eur") > 0)
).approxQuantile("market_value_in_eur", [0.5], 0.01)[0]  # 0.01 es la precisión

print(f"Mediana del valor de mercado (excluyendo valores 0): {mediana_market_value}")

# Sustituir nulos y ceros por 'mediana_market_value'
df_players = df_players.withColumn(
    "market_value_in_eur",
    when(col("market_value_in_eur").isNull() | (col("market_value_in_eur") == 0), mediana_market_value)
    .otherwise(col("market_value_in_eur"))
)

# Comprobar de nuevo los nulos y valores minimos despues de la sustitucion
contar_nulos_por_columna(df_players).show()
df_players.select('market_value_in_eur').summary('min').show()

In [ ]:
# Repetimos el proceso para la columna 'highest_market_value_in_eur'

# Calcular la mediana del valor de mercado (excluyendo nulos y ceros)
mediana_market_value = df_players.filter(
    col("highest_market_value_in_eur").isNotNull() & (col("highest_market_value_in_eur") > 0)
).approxQuantile("highest_market_value_in_eur", [0.5], 0.01)[0]  # 0.01 es la precisión

print(f"Mediana del valor máximo de mercado (excluyendo valores 0): {mediana_market_value}")

# Sustituir nulos y ceros por 'mediana_market_value'
df_players = df_players.withColumn(
    "highest_market_value_in_eur",
    when(col("highest_market_value_in_eur").isNull() | (col("highest_market_value_in_eur") == 0), mediana_market_value)
    .otherwise(col("highest_market_value_in_eur"))
)

# Comprobar de nuevo los nulos y valores minimos despues de la sustitucion
contar_nulos_por_columna(df_players).show()
df_players.select('highest_market_value_in_eur').summary('min').show()

In [ ]:
# Ver nulos en la columna 'height_in_cm'
df_players.filter(col('height_in_cm').isNull()).show()

In [ ]:
# Ver el valor minimo de la columna 'height_in_cm'
df_players.select('height_in_cm').summary('min').show()

In [ ]:
# El valor minimo de altura es 17cm, lo cual es imposible
# Sustituimos los valores nulos y minimos por la media de altura

# Calcular la media de altura (excluyendo nulos y valores <= 100)
avg_height = df_players.filter(col("height_in_cm").isNotNull()).filter(col("height_in_cm") > 100).agg(avg("height_in_cm")).collect()[0][0]
print(f"Media de la altura (excluyendo valores <= 100): {avg_height}")

# Sustituimos los valores nulos y minimos por 'avg_height'
df_players = df_players.withColumn(
    "height_in_cm",
    when(col("height_in_cm").isNull() | (col("height_in_cm") <= 100), avg_height).otherwise(col("height_in_cm"))
)

# Comprobar de nuevo los nulos y valores minimos despues de la sustitucion
contar_nulos_por_columna(df_players).show()
df_players.select('height_in_cm').summary('min').show()

In [ ]:
# Ver nulos en la columna 'first_name'
df_players.filter(col('first_name').isNull()).show()

In [ ]:
# Parece que los nulos de la columna 'first_name' se deben a que el jugador aparece con su apodo
# Sustituir los first_name nulos por 'Desconocido'

df_players = df_players.withColumn(
    "first_name",
    when(col("first_name").isNull(), "desconocido").otherwise(col("first_name"))
)
# Ver nulos de nuevo
contar_nulos_por_columna(df_players).show()

In [ ]:
# Ver nulos de la columna 'date_of_birth'
df_players.filter(col('date_of_birth').isNull()).show()

In [ ]:
# Sustituimos las fechas de nacimiento nulas por un valor por defecto
# para indicar que la fecha es desconocida (no deberia ser una fecha que pueda aparecer en otros registros)
default_timestamp = to_timestamp(lit('1900-01-01 00:00:00'))

df_players = df_players.withColumn(
    "date_of_birth",
    when(col("date_of_birth").isNull(), default_timestamp).otherwise(col("date_of_birth"))
)
# Ver nulos de nuevo
contar_nulos_por_columna(df_players).show()

In [ ]:
# Ver nulos de la columna 'sub_position'
df_players.filter(col('sub_position').isNull()).show()

In [ ]:
# En la columna 'position' ya existe un valor 'Missing' que podemos usar para corregir los nulos
df_players = df_players.withColumn(
    "sub_position",
    when(col("sub_position").isNull(), "Missing").otherwise(col("sub_position"))
)
# Ver nulos de nuevo
contar_nulos_por_columna(df_players).show()

In [ ]:
# Ver nulos de las columnas 'country_of_birth' y 'city_of_birth'
df_players.filter(col('country_of_birth').isNull()).show()
df_players.filter(col('city_of_birth').isNull()).show()

En las columnas 'country_of_birth' y 'city_of_birth' hay varios casos donde no hay valor para el país pero si para la ciudad. Para recuperar esos datos perdidos, definimos una función que mapea ciudad a país usando `geopy`

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time

# Instanciamos el geolocalizador
geolocator = Nominatim(user_agent="city_to_country_mapper")

# Función para mapear ciudad a país
def get_country(city):
    try:
        location = geolocator.geocode(city, language="en", timeout=10)
        if location and location.address:
            address_parts = location.address.split(", ")
            for part in reversed(address_parts):
                if len(part) > 2 and not any(char.isdigit() for char in part):
                    return part
        return None
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Error geocoding {city}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error for {city}: {e}")
        return None

# Crea una UDF para usar en Spark
get_country_udf = udf(get_country, StringType())

# Filtra las ciudades donde no hay pais pero hay ciudad
cities_df = df_players.filter(col("country_of_birth").isNull() & col("city_of_birth").isNotNull()) \
                   .select("city_of_birth").distinct()

# Collect para traerlas al driver (Cuidado si hay muchas)
cities = [row['city_of_birth'] for row in cities_df.collect() if row['city_of_birth'] is not None]

# Diccionario ciudad-país
city_country_map = {}

for city in cities:
    country = get_country(city)
    if country:
        city_country_map[city] = country
        print(f"Mapped {city} to {country}")
    else:
        city_country_map[city] = None
        print(f"Could not map {city}")
    time.sleep(1)  # No bombardees el server como un cavernícola

# Transforma el diccionario a un DataFrame para hacer join
mapping_rows = [(k, v) for k, v in city_country_map.items()]
mapping_df = spark.createDataFrame(mapping_rows, ["city_of_birth", "mapped_country"])

# Une el mapping con el DataFrame original
df_players_mapped = df_players.join(mapping_df, on="city_of_birth", how="left")

# Actualiza la columna 'country_of_birth' donde sea NULL
df_players_mapped = df_players_mapped.withColumn("country_of_birth",
    when(col("country_of_birth").isNull() & col("mapped_country").isNotNull(), col("mapped_country"))
    .otherwise(col("country_of_birth"))
)

# (Opcional) Quita la columna de mapeo
df_players_mapped = df_players_mapped.drop("mapped_country")

In [ ]:
# Comprobar si el mapeo se ha realizado correctamente
print(f"Valores nulos en 'country_of_birth' antes del mapeo:", df_players.filter(col('country_of_birth').isNull()).count())
print(f"Valores nulos en 'country_of_birth' despues del mapeo:", df_players_mapped.filter(col('country_of_birth').isNull()).count())

In [ ]:
# El resto de valores nulos son de filas donde no hay ni pais ni ciudad
# Sustituimos esos valores por un valor 'Desconocido'
df_players_mapped = df_players_mapped.withColumn(
    "country_of_birth",
    when(col("country_of_birth").isNull(), "Desconocido").otherwise(col("country_of_birth"))
)
df_players_mapped = df_players_mapped.withColumn(
    "city_of_birth",
    when(col("city_of_birth").isNull(), "Desconocida").otherwise(col("city_of_birth"))
)
# Ver nulos de nuevo
contar_nulos_por_columna(df_players_mapped).show()

In [ ]:
# Eliminamos las columnas 'country_of_citizenship', 'image_url' y 'url' porque no son utiles para nuestro analisis
df_players_mapped = df_players_mapped.drop("country_of_citizenship","image_url", "url")
df_players_mapped.show()

# Archivo "transfers.csv"

In [ ]:
# Cargar el csv
df_transfers = spark.read.csv("/root/.cache/kagglehub/datasets/davidcariboo/player-scores/versions/602/transfers.csv", header=True, inferSchema=True)

# Ver las primeras filas
df_transfers.show(10)
df_transfers.printSchema()
df_transfers.columns

In [ ]:
mostrar_sumario(df_transfers).show()

In [ ]:
contar_nulos_por_columna(df_transfers).show()

In [ ]:
duplicados_por_columna(df_transfers, "player_id")

In [ ]:
# Sustituir los nulos de la columna transfer_fee por 0

df_transfers = df_transfers.withColumn(
    "transfer_fee",
    when(col("transfer_fee").isNull(), 0).otherwise(col("transfer_fee"))
)

# Comprobar de nuevo los nulos despues de la sustitucion
contar_nulos_por_columna(df_transfers).show()


In [ ]:
# Sustituir los nulos de market_value_in_eur por la mediana del equipo que procede

# Calcular la mediana del valor de mercado por jugador (excluyendo nulos y ceros)
mediana_market_value_por_jugador = df_transfers.filter(
    col("market_value_in_eur").isNotNull() & (col("market_value_in_eur") > 0)
).groupBy("player_id").agg(
    approx_percentile("market_value_in_eur", 0.5).alias("mediana_mv_por_jugador")
)

# Unir la mediana por club al DataFrame original
df_transfers = df_transfers.join(mediana_market_value_por_jugador, on="player_id", how="left_outer")

# Sustituir nulos en 'market_value_in_eur' por la mediana del jugador
df_transfers = df_transfers.withColumn(
    "market_value_in_eur",
    when(col("market_value_in_eur").isNull(), col("mediana_mv_por_jugador")).otherwise(col("market_value_in_eur"))
)

# Eliminar la columna temporal de la mediana por club
df_transfers = df_transfers.drop("mediana_mv_por_jugador")

# Comprobar de nuevo los nulos despues de la sustitucion
contar_nulos_por_columna(df_transfers).show()
df_transfers.select("player_id", "market_value_in_eur", "transfer_fee").show(20)

In [ ]:
# Aun quedan nulos en 'market_value_in_eur', estos son de jugadores que no estaban en 'df_players' o no tenían market_value > 0
# Podemos rellenarlos con la mediana general
mediana_market_value = df_transfers.filter(
    col("market_value_in_eur").isNotNull()
).agg(approx_percentile("market_value_in_eur", 0.5)).collect()[0][0]

df_transfers = df_transfers.withColumn(
    "market_value_in_eur",
    when(col("market_value_in_eur").isNull(), mediana_market_value).otherwise(col("market_value_in_eur"))
)
# Comprobar de nuevo los nulos despues de la sustitucion
contar_nulos_por_columna(df_transfers).show()
df_transfers.select("player_id", "market_value_in_eur", "transfer_fee").show(20)

# Guardar las tablas limpias

In [ ]:
# Ruta donde guardar el archivo CSV
output_path = "../Data"

# Guardar cada DataFrame en formato CSV
# Spark por defecto guarda en particiones
# Para guardarlo en un solo archivo, usar repartition(1) antes de 'write'
df_clubs.repartition(1).write.csv(output_path + "/clubs", header=True, mode="overwrite")
df_competitions.repartition(1).write.csv(output_path + "/competitions", header=True, mode="overwrite")
df_game_events.repartition(1).write.csv(output_path + "/game_events", header=True, mode="overwrite")
df_game_lineups_cleaned.repartition(1).write.csv(output_path + "/game_lineups", header=True, mode="overwrite")
df_appearances.repartition(1).write.csv(output_path + "/appearances", header=True, mode="overwrite")
df_player_valuations.repartition(1).write.csv(output_path + "/player_valuations", header=True, mode="overwrite")
df_games.repartition(1).write.csv(output_path + "/games", header=True, mode="overwrite")
df_players_mapped.repartition(1).write.csv(output_path + "/players", header=True, mode="overwrite")
df_transfers.repartition(1).write.csv(output_path + "/transfers", header=True, mode="overwrite")

In [ ]:
# Importante cerrar la sesion de spark
spark.stop()